In [0]:
#Importing necessary libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,col, count,desc,when,concat,lit
spark=SparkSession.builder.appName('Pyspark Practice').getOrCreate()

In [0]:
#Loading File
Filepath='dbfs:/FileStore/bank.csv'

df=spark.read.csv(Filepath,header=True, inferSchema=True)
#df.describe()
#df.printSchema()
df.show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
| 56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
| 41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
| 55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
| 54|     admin.| married| tertiary|     no|    184|     no|  

In [0]:
#Apply some dataframe functions
#df_groupby=df.groupBy("deposit").agg(count("*").alias("count"))
df_groupby=df.groupBy("job","marital").agg(sum("balance").alias("sum_of_balance"))
df_orderby=df_groupby.orderBy(desc("sum_of_balance"))
df_orderby.show()

+-------------+--------+--------------+
|          job| marital|sum_of_balance|
+-------------+--------+--------------+
|   management| married|       2561712|
|  blue-collar| married|       1722351|
|   management|  single|       1530268|
|      retired| married|       1526151|
|   technician| married|       1462384|
|   technician|  single|       1140197|
|       admin.| married|        822432|
|     services| married|        663676|
|       admin.|  single|        585175|
|      student|  single|        525091|
|   management|divorced|        510561|
|  blue-collar|  single|        484383|
| entrepreneur| married|        436422|
|self-employed| married|        351616|
|      retired|divorced|        319844|
|   unemployed| married|        254991|
|     services|  single|        254178|
|self-employed|  single|        249471|
|    housemaid| married|        238777|
|   technician|divorced|        234544|
+-------------+--------+--------------+
only showing top 20 rows



In [0]:
#Get summary of each column
df_summary=df.describe()
#df_summary.show()
pandas_df=df_summary.toPandas().transpose()
print(pandas_df)

               0                   1                   2         3        4
summary    count                mean              stddev       min      max
age        11162  41.231947679627304  11.913369192215518        18       95
job        11162                None                None    admin.  unknown
marital    11162                None                None  divorced   single
education  11162                None                None   primary  unknown
default    11162                None                None        no      yes
balance    11162  1528.5385235620856   3225.413325946149     -6847    81204
housing    11162                None                None        no      yes
loan       11162                None                None        no      yes
contact    11162                None                None  cellular  unknown
day        11162  15.658036194230425   8.420739541006462         1       31
month      11162                None                None       apr      sep
duration   1

In [0]:
#to check null and nan values
#df.printSchema()
df_null_nan=df.select(*[when(col(c).isNull(),True).otherwise(False).alias(c + "_isnull") for c in df.columns])
pandasdf=df_null_nan.toPandas()
#print(pandasdf)

#to get a summary of non null and null values

df_null_nonnull_summ=df.agg(*[sum(when(col(c).isNotNull(),1).otherwise(0)).alias(c + "_isnull") for c in df.columns])
df_null_nonnull_summ_pandas=df_null_nonnull_summ.toPandas()
df_null_nonnull_summ_pandas

,age_isnull,job_isnull,marital_isnull,education_isnull,default_isnull,balance_isnull,housing_isnull,loan_isnull,contact_isnull,day_isnull,month_isnull,duration_isnull,campaign_isnull,pdays_isnull,previous_isnull,poutcome_isnull,deposit_isnull
0,11162,11162,11162,11162,11162,11162,11162,11162,11162,11162,11162,11162,11162,11162,11162,11162,11162


In [0]:
#sum_of_bal=df.agg(sum(col("balance")))

df_filter=df.filter(col("age")<30)

#df_filter.show()

#df_filter.count()

df_filter.toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9,...,1541,1542,1543,1544,1545,1546,1547,1548,1549,1550
age,28,29,28,28,26,23,29,26,29,24,...,25,26,28,27,26,28,25,25,27,26
job,services,management,admin.,blue-collar,blue-collar,entrepreneur,management,blue-collar,admin.,technician,...,technician,services,technician,services,services,technician,services,services,management,admin.
marital,single,married,divorced,single,single,single,married,married,single,single,...,single,single,single,single,single,single,married,single,single,married
education,secondary,tertiary,secondary,secondary,secondary,primary,tertiary,secondary,secondary,secondary,...,secondary,secondary,secondary,secondary,secondary,secondary,secondary,secondary,tertiary,secondary
default,no,no,no,no,no,no,no,no,no,no,...,no,no,no,no,no,no,no,no,no,no
balance,5090,199,785,759,82,4,10576,246,163,409,...,173,531,3,139,162,2000,50,199,390,714
housing,yes,yes,yes,yes,yes,yes,no,yes,yes,yes,...,yes,yes,no,no,yes,no,yes,no,no,yes
loan,no,yes,no,no,no,no,no,no,yes,no,...,no,yes,no,no,no,no,no,no,no,no
contact,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,...,cellular,cellular,telephone,cellular,cellular,telephone,unknown,cellular,cellular,unknown
day,6,7,8,9,9,13,15,16,16,16,...,15,6,18,25,2,3,5,16,28,28


In [0]:
#select all columns and copy a dataframe

df_select_all=df.select("*")

df_select_all.show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
| 56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
| 41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
| 55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
| 54|     admin.| married| tertiary|     no|    184|     no|  

In [0]:
#adding a new column in a datafrme

new_column_exm_1=df_select_all.withColumn('new_column',df_select_all.age)
#new_column_exm_1.show()

#alternate method to add new column

new_column_exm_2=df_select_all.withColumn('new_column',col("job"))
#new_column_exm_2.show()

#adding new column by concatenation of 2 columns

new_column_exm_3=df_select_all.withColumn('new_column',concat(col('age'),lit("_"),col('job')))
new_column_exm_3.show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+--------------+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|    new_column|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+--------------+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|     59_admin.|
| 56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|     56_admin.|
| 41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes| 41_technician|
| 55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1

In [0]:
#dropping a column from a dataframe
df_copy=df.select("*")

df_droped_col=df_copy.drop(col("age"))
df_droped_col.show()

+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
|     admin.| married| tertiary|     no|    184|     no|  no|unknown|  5|  may|     673|  

In [0]:
#Rename the column name

df_rename=df_copy.withColumnRenamed('age','age_new')
df_rename.show()

#example of how pyspark dataframes are immutable
df_copy.withColumnRenamed('age','age_new')
#df_copy.show()

+-------+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age_new|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+-------+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|     59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
|     56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
|     41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
|     55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
|     54|     admin.| married| ter

In [0]:
#get distinct rows

#df_copy.count()

df_copy.distinct().count()

Out[39]: 11162

In [0]:
#apply limit to get sample

df_copy.limit(15).show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
| 56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
| 41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
| 55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
| 54|     admin.| married| tertiary|     no|    184|     no|  

In [0]:
#union

df_copy.union(df_copy).count()

Out[47]: 22324

In [0]:
#join
df_join1=df.alias('df1')

df_join2=df.alias('df2')

df_final_join=df_join1.join(df_join2,df_join1['job']==df_join2['job'],'inner')

df_final_join.show()

+---+------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+---+------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+-------+
|age|   job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|age|   job| marital|education|default|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+---+------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|admin.|married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes| 43|admin.|  single|secondary|     no|     35|     no|  no|telephone|  9|  nov|     208|       1|   -1

In [0]:
#sampling a dataframe

df_sample=df_copy.sample(withReplacement=True,fraction=0.2)
df_sample.count()

Out[66]: 2276

In [0]:
#Pivot example

df_pivot=df_copy.groupBy('age').pivot('job').sum('age')
df_pivot.show()

+---+------+-----------+------------+---------+----------+-------+-------------+--------+-------+----------+----------+-------+
|age|admin.|blue-collar|entrepreneur|housemaid|management|retired|self-employed|services|student|technician|unemployed|unknown|
+---+------+-----------+------------+---------+----------+-------+-------------+--------+-------+----------+----------+-------+
| 31|  1829|       2511|         372|      124|      4402|   null|          713|    1395|    310|      2945|       682|     93|
| 85|  null|       null|        null|     null|      null|    425|         null|    null|   null|      null|      null|   null|
| 65|   130|       null|         130|       65|       195|   1105|         null|    null|   null|      null|        65|     65|
| 53|  1219|       2067|         424|      530|      2544|    530|          318|     742|   null|      2014|       265|    159|
| 78|  null|       null|        null|     null|        78|   1326|         null|    null|   null|      n

In [0]:
#drop duplicates

df_drop_dups=df_copy.drop_duplicates()
#df_drop_dups.count()

df_count_of_age=df_copy.select('age').count()
#df_count_of_age

#print(df_count_of_age)

#df_count_of_age=df_copy.groupBy('age').count()
#df_count_of_age.show()

list=['age']
df_drop_dups_spec=df_copy.drop_duplicates(list)
#df_drop_dups_spec.count()

11162
